# Denoising Diffusion Probabilistic Models

In [ ]:
# Standard library imports
import math
import os
import time
import shutil
import logging
from collections.abc import Mapping
from pathlib import Path
from operator import attrgetter, itemgetter
from functools import partial
from copy import copy
from contextlib import contextmanager

# Third-party library imports
import fastcore.all as fc
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import torch
import random
import gzip
import pickle

# torch imports
import torchvision.transforms.functional as TF
import torch.nn.functional as F
from torch import tensor, nn, optim
from torch.utils.data import DataLoader, default_collate
from torch.nn import init
from torch.optim import lr_scheduler
from torcheval.metrics import MulticlassAccuracy

# dataset imports
import datasets
from datasets import load_dataset, load_dataset_builder

# miniai imports
from miniai.datasets import *
from miniai.conv import *
from miniai.learner import *
from miniai.activations import *
from miniai.init import *
from miniai.sgd import *
from miniai.resnet import *
from miniai.augment import *

In [ ]:
mpl.rcParams['image.cmap'] = 'gray'
logging.disable(logging.WARNING)

In [ ]:
x,y = 'image', 'label'
dsd = load_dataset('fashion_mnist')

In [ ]:
@inplace
def transformi(b):
    b[x] = [TF.resize(TF.to_tensor(o), (32,32)) for o in b[x]] # resize the 28x28 images to 32x32 to make it simpler for model's architecture 

In [ ]:
set_seed(42)
bs = 128
tds = dsd.with_transform(transformi)
dls = DataLoaders.from_dd(tds, bs, num_workers=8)
dt = dls.train
xb,yb = next(iter(dt))
xb.shape,yb[:10]